In [35]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_jyu25" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]
my_char='r'


beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

source_url = "https://sourceforge.net/directory/?q=" + my_char + "&sort=name&page="
rest_url = "https://sourceforge.net/rest/p/"

gleft = 0

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def get_source(url, coll, rest):
    page = 1
    project_count = 0
    while True:
        resp = requests.get(url + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        if re.search('No results found.', soup.get_text()):
            return

        for link in soup.find_all(class_="project-icon", href=True):
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            name = name[0] if name else None
            if name is not None and name.lower().startswith(my_char):
                resp = requests.get(rest + name)
                if resp.status_code == 200:
                    info = json.loads(resp.text)
                    info['forge'] = 'sourceforge'
                    print(info)
                    coll.insert_one(info)
                    project_count += 1
                    if project_count >= 50:
                        print('Finish!')
                        return
        page += 1
    return

# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):
            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            print('Start!')
            for el in array:
                temp = re.findall(r'\'+\w+\'', str(el))
                if temp[1][1] == 'r' or temp[1][1] == 'R':
                    print(el)
                    size+=1
                    el['forge'] = 'gitlab'
                    coll.insert(el)
                
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            temp = re.findall(r'\'+\w+\'', str(el))
                            if temp[1][1] == 'r' or temp[1][1] == 'R':
                                print(el)
                                size+=1
                                el['forge'] = 'gitlab'
                                coll.insert(el)
                                if size >= 50:
                                    return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
#start retrieving        
get(beginurl,coll)
get_source(source_url, coll, rest_url)

Start!
{'path': 'roadmap', 'tag_list': [], 'readme_url': None, 'star_count': 0, 'http_url_to_repo': 'https://gitlab.com/nodeferret/roadmap.git', 'name_with_namespace': 'nodeferret / Nodeferret Roadmap', 'namespace': {'path': 'nodeferret', 'parent_id': None, 'name': 'nodeferret', 'id': 3939117, 'full_path': 'nodeferret', 'kind': 'group'}, 'name': 'Nodeferret Roadmap', 'web_url': 'https://gitlab.com/nodeferret/roadmap', 'avatar_url': None, 'description': '', 'ssh_url_to_repo': 'git@gitlab.com:nodeferret/roadmap.git', 'default_branch': None, 'created_at': '2018-11-04T20:54:59.653Z', 'forks_count': 0, 'id': 9210906, 'path_with_namespace': 'nodeferret/roadmap', 'last_activity_at': '2018-11-04T20:54:59.653Z'}


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{'path': 'rev', 'tag_list': [], 'readme_url': None, 'star_count': 0, 'http_url_to_repo': 'https://gitlab.com/csgowl/rev.git', 'name_with_namespace': 'Pavankumar Gollapudi / rev', 'namespace': {'path': 'csgowl', 'parent_id': None, 'name': 'csgowl', 'id': 3938828, 'full_path': 'csgowl', 'kind': 'user'}, 'name': 'rev', 'web_url': 'https://gitlab.com/csgowl/rev', 'avatar_url': None, 'description': '', 'ssh_url_to_repo': 'git@gitlab.com:csgowl/rev.git', 'default_branch': 'master', 'created_at': '2018-11-04T19:23:46.261Z', 'forks_count': 0, 'id': 9210097, 'path_with_namespace': 'csgowl/rev', 'last_activity_at': '2018-11-04T21:42:14.302Z'}


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:122: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{'path': 'rules', 'tag_list': [], 'readme_url': None, 'star_count': 0, 'http_url_to_repo': 'https://gitlab.com/robosoccer/rules.git', 'name_with_namespace': 'robosoccer / Rules', 'namespace': {'path': 'robosoccer', 'parent_id': None, 'name': 'robosoccer', 'id': 3938675, 'full_path': 'robosoccer', 'kind': 'group'}, 'name': 'Rules', 'web_url': 'https://gitlab.com/robosoccer/rules', 'avatar_url': None, 'description': '', 'ssh_url_to_repo': 'git@gitlab.com:robosoccer/rules.git', 'default_branch': 'master', 'created_at': '2018-11-04T18:36:20.798Z', 'forks_count': 0, 'id': 9209660, 'path_with_namespace': 'robosoccer/rules', 'last_activity_at': '2018-11-04T19:52:19.176Z'}
{'path': 'redirector', 'tag_list': [], 'readme_url': 'https://gitlab.com/yzguy/redirector/blob/master/README.md', 'star_count': 0, 'http_url_to_repo': 'https://gitlab.com/yzguy/redirector.git', 'name_with_namespace': 'Adam Smith / redirector', 'namespace': {'path': 'yzguy', 'parent_id': None, 'name': 'yzguy', 'id': 1280792, 

...

{'screenshots': [], 'shortname': 'randr', 'developers': [{'name': 'James David Low', 'username': 'musicmanj', 'url': 'https://sourceforge.net/u/musicmanj/'}], 'name': 'R and R', 'preferred_support_url': '', 'short_description': 'R and R - short for REST and Relaxation, is REALLY simple REST client / server offering compatible libraries in PHP and Java and other languages to come.\r\n\r\nThis project has been moved to: https://github.com/jamesdlow/randr', 'socialnetworks': [{'accounturl': '', 'socialnetwork': 'Twitter'}, {'accounturl': None, 'socialnetwork': 'Facebook'}], 'private': False, 'external_homepage': 'https://github.com/jamesdlow/randr', 'forge': 'sourceforge', 'categories': {'language': [{'fullpath': 'Programming Language :: PHP', 'fullname': 'PHP', 'id': 183, 'shortname': 'php'}, {'fullpath': 'Programming Language :: Java', 'fullname': 'Java', 'id': 198, 'shortname': 'java'}], 'environment': [{'fullpath': 'User Interface :: Web-based', 'fullname': 'Web-based', 'id': 237, 'sh

{'screenshots': [], 'shortname': 'rpackagehydro', 'developers': [{'name': 'reneca', 'username': 'reneca', 'url': 'https://sourceforge.net/u/reneca/'}, {'name': 'chris birkel', 'username': 'birkel', 'url': 'https://sourceforge.net/u/birkel/'}], 'name': 'R package for simple hydrological models', 'preferred_support_url': '', 'short_description': 'A set of R functions (www.r-project.org) for simple hydrological models, based on linear and non-linear storages, MC simulation and multi-objective evaluation. The code provided here is aimed to be transformed into a proper R package at some point.', 'socialnetworks': [], 'private': False, 'external_homepage': 'https://rpackagehydro.sourceforge.io', 'forge': 'sourceforge', 'categories': {'language': [{'fullpath': 'Programming Language :: S/R', 'fullname': 'S/R', 'id': 573, 'shortname': 's_slash_r'}], 'environment': [{'fullpath': 'User Interface :: Textual :: Command-line', 'fullname': 'Command-line', 'id': 459, 'shortname': 'ui_commandline'}], '

{'screenshots': [], 'shortname': 'rscriptsmultivariate', 'developers': [{'name': 'Aleksander Mendyk', 'username': 'olenzor', 'url': 'https://sourceforge.net/u/olenzor/'}, {'name': 'Jakub Szlęk', 'username': 'jszlek', 'url': 'https://sourceforge.net/u/jszlek/'}, {'name': 'pezhman', 'username': 'pkazemi', 'url': 'https://sourceforge.net/u/pkazemi/'}, {'name': 'Adam Pacławski', 'username': 'adampaclawski', 'url': 'https://sourceforge.net/u/adampaclawski/'}, {'name': 'Hassan Khalid', 'username': 'mrhash101', 'url': 'https://sourceforge.net/u/mrhash101/'}], 'name': 'R scripts for multivariate analysis', 'preferred_support_url': '', 'short_description': '', 'socialnetworks': [], 'private': False, 'external_homepage': '', 'forge': 'sourceforge', 'categories': {'language': [], 'environment': [], 'license': [], 'topic': [], 'database': [], 'translation': [], 'audience': [], 'os': [], 'developmentstatus': []}, 'status': 'active', 'url': 'https://sourceforge.net/p/rscriptsmultivariate/', 'video_u

{'screenshots': [], 'shortname': 'revinsp', 'developers': [{'name': 'stefano ravegnani', 'username': 'sravegnani', 'url': 'https://sourceforge.net/u/sravegnani/'}], 'name': 'R&I', 'preferred_support_url': '', 'short_description': 'review and inspection improvement process', 'socialnetworks': [], 'private': False, 'external_homepage': 'https://revinsp.sourceforge.io', 'forge': 'sourceforge', 'categories': {'language': [], 'environment': [], 'license': [], 'topic': [], 'database': [], 'translation': [], 'audience': [], 'os': [], 'developmentstatus': []}, 'status': 'active', 'url': 'https://sourceforge.net/p/revinsp/', 'video_url': '', 'creation_date': '2010-10-06', 'preferred_support_tool': '', 'summary': '', 'icon_url': None, '_id': '5166dedbe88f3d3659d5a799', 'labels': [], 'tools': [{'installable': False, 'url': '/p/revinsp/reviews/', 'tool_label': 'Reviews', 'name': 'reviews', 'mount_label': 'Reviews', 'icons': {'32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blo

{'screenshots': [], 'shortname': 'rrdevtemp', 'developers': [{'name': 'TheIntercooler', 'username': 'theintercooler', 'url': 'https://sourceforge.net/u/theintercooler/'}, {'name': 'SfcoAwol', 'username': 'sfcoawol', 'url': 'https://sourceforge.net/u/sfcoawol/'}, {'name': 'Memorix101', 'username': 'memorix101', 'url': 'https://sourceforge.net/u/memorix101/'}, {'name': 'void*foo', 'username': 'void-foo', 'url': 'https://sourceforge.net/u/void-foo/'}, {'name': 'Leviscus Tempris', 'username': 'tempris', 'url': 'https://sourceforge.net/u/tempris/'}], 'name': 'R+RDevTEMP', 'preferred_support_url': 'http://theintercooler.com/forum-3?mingleforumaction=vforum&g=5.0', 'short_description': '', 'socialnetworks': [{'accounturl': '', 'socialnetwork': 'Twitter'}, {'accounturl': None, 'socialnetwork': 'Facebook'}], 'private': False, 'external_homepage': 'http://theintercooler.com/reveladerevolution', 'forge': 'sourceforge', 'categories': {'language': [], 'environment': [], 'license': [], 'topic': [], 